In [1]:
import torch

In [2]:
print(torch.__version__)

2.4.0+cu121


In [3]:
print(torch.cuda.is_available())

True


In [19]:
some_tensor = torch.tensor([])
print(some_tensor, some_tensor.dtype)
tensor0d = torch.tensor(1)
tensor1d = torch.tensor([1, 2, 3])
tensor2d = torch.tensor([[1, 2], [3, 4]])
tensor3d = torch.tensor([[[1, 2], [3, 4]], [[5, 6], [7, 8]]])

tensor([]) torch.float32


In [15]:
print(tensor0d.dtype)
print(tensor1d.dtype)
print(tensor2d.dtype)
floatvec = torch.tensor([1.15,2.2,3.71])
print(floatvec.dtype)
floatvec = tensor1d.to(torch.float32)
print(floatvec.dtype)
floatvec = floatvec.to(torch.int64)
print(floatvec, floatvec.dtype)

torch.int64
torch.int64
torch.int64
torch.float32
torch.float32
tensor([1, 2, 3]) torch.int64


In [20]:
# Common PyTorch operations
tensor2d = torch.tensor([[1,2,3],[4,5,6]])
print(tensor2d)
print(tensor2d.shape)
print(tensor2d.reshape(3,2))
print(tensor2d.view(3,2)) # Requires original data to be contiguous
print(tensor2d.T)
print(tensor2d.matmul(tensor2d.T))
print(tensor2d @ tensor2d.T)

tensor([[1, 2, 3],
        [4, 5, 6]])
torch.Size([2, 3])
tensor([[1, 2],
        [3, 4],
        [5, 6]])
tensor([[1, 2],
        [3, 4],
        [5, 6]])
tensor([[1, 4],
        [2, 5],
        [3, 6]])
tensor([[14, 32],
        [32, 77]])
tensor([[14, 32],
        [32, 77]])


In [ ]:
t2d = tensor2d.T
print(t2d)
print(t2d.reshape(2,3))
# print(t2d.view(2,3))

tensor([[1, 4],
        [2, 5],
        [3, 6]])
tensor([[1, 4, 2],
        [5, 3, 6]])


RuntimeError: view size is not compatible with input tensor's size and stride (at least one dimension spans across two contiguous subspaces). Use .reshape(...) instead.

In [8]:
import torch.nn.functional as F

In [9]:
# Logistic Regression Forward Pass
y = torch.tensor([1.0])
x1 = torch.tensor([1.1])
w1 = torch.tensor([2.2])
b = torch.tensor([0.0])
z = x1 * w1 + b
a = torch.sigmoid(z)
loss = F.binary_cross_entropy(a, y)
# autograd will create a computation graph automatically if one of its terminal nodes has requires_grad=True.

In [10]:
from torch.autograd import grad

In [11]:
# Computing gradient using autograd
w1 = torch.tensor([2.2], requires_grad=True)
b = torch.tensor([0.0], requires_grad=True)
z = x1 * w1 + b
a = torch.sigmoid(z)
loss = F.binary_cross_entropy(a, y)
grad_L_w1 = grad(loss, w1, retain_graph=True) # Retain graph otherwise detroyed to save memory
grad_L_b = grad(loss, b, retain_graph=True)
print(grad_L_w1)
print(grad_L_b)
loss.backward()
print(w1.grad)
print(b.grad)

(tensor([-0.0898]),)
(tensor([-0.0817]),)
tensor([-0.0898])
tensor([-0.0817])


In [12]:
# A multilayer perceptron with two hidden layers
class NeuralNetwork(torch.nn.Module):
    def __init__(self, num_inputs, num_outputs):
        super().__init__()
        self.layers = torch.nn.Sequential(
            torch.nn.Linear(num_inputs, 30), # Feed forward / Fully connected layers
            torch.nn.ReLU(),
            torch.nn.Linear(30,20),
            torch.nn.ReLU(),
            torch.nn.Linear(20, num_outputs)
        )

    def forward(self, x):
        logits = self.layers(x)
        return logits
    
model = NeuralNetwork(50, 3)
print(model)

NeuralNetwork(
  (layers): Sequential(
    (0): Linear(in_features=50, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=3, bias=True)
  )
)


In [13]:
# Total number of trainable model parameters
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(num_params)
# Weights
print(model.layers[0].weight)
print(model.layers[0].weight.shape)
print(model.layers[0].bias)

2213
Parameter containing:
tensor([[ 0.1116,  0.1056, -0.0756,  ...,  0.0856, -0.1105, -0.0821],
        [ 0.1202, -0.0847, -0.0163,  ..., -0.1027,  0.0763,  0.0280],
        [ 0.0139, -0.0357,  0.0094,  ...,  0.0447,  0.1058, -0.0943],
        ...,
        [ 0.0063,  0.0117, -0.1390,  ..., -0.0337,  0.0861, -0.0548],
        [-0.0797, -0.0044, -0.0114,  ..., -0.0574, -0.0425,  0.1145],
        [-0.0945,  0.0284,  0.0482,  ..., -0.0601,  0.0734, -0.1211]],
       requires_grad=True)
torch.Size([30, 50])
Parameter containing:
tensor([-0.0006, -0.1017, -0.0948, -0.0628, -0.0627,  0.0939, -0.0165,  0.0184,
        -0.0825,  0.0005, -0.0298, -0.0074, -0.0070,  0.0855,  0.0678,  0.0243,
         0.0616,  0.0406, -0.0385, -0.1053, -0.0039,  0.0822, -0.0634,  0.1276,
         0.0689,  0.0831, -0.1168,  0.1296,  0.1015,  0.1344],
       requires_grad=True)


In [14]:
torch.manual_seed(123)
model = NeuralNetwork(50, 3)
print(model.layers[0].weight)
X = torch.rand((1,50))
out = model(X)
print(out)

Parameter containing:
tensor([[-0.0577,  0.0047, -0.0702,  ...,  0.0222,  0.1260,  0.0865],
        [ 0.0502,  0.0307,  0.0333,  ...,  0.0951,  0.1134, -0.0297],
        [ 0.1077, -0.1108,  0.0122,  ...,  0.0108, -0.1049, -0.1063],
        ...,
        [-0.0787,  0.1259,  0.0803,  ...,  0.1218,  0.1303, -0.1351],
        [ 0.1359,  0.0175, -0.0673,  ...,  0.0674,  0.0676,  0.1058],
        [ 0.0790,  0.1343, -0.0293,  ...,  0.0344, -0.0971, -0.0509]],
       requires_grad=True)
tensor([[-0.1670,  0.1001, -0.1219]], grad_fn=<AddmmBackward0>)


In [15]:
with torch.no_grad():
    out = model(X)
print(out)

tensor([[-0.1670,  0.1001, -0.1219]])


In [16]:
X_train = torch.tensor([
    [-1.2, 3.1],
    [-0.9, 2.9],
    [-0.5, 2.6],
    [2.3, -1.1],
    [2.7, -1.5]
])
y_train = torch.tensor([0, 0, 0, 1, 1])
X_test = torch.tensor([
    [-0.8, 2.8],
    [2.6, -1.6],
])
y_test = torch.tensor([0, 1])

In [17]:
from torch.utils.data import Dataset
class ToyDataset(Dataset):
    def __init__(self, X, y):
        self.features = X
        self.labels = y
    def __getitem__(self, index):       
        one_x = self.features[index]    
        one_y = self.labels[index]      
        return one_x, one_y             
    def __len__(self): 
        return self.labels.shape[0]     
train_ds = ToyDataset(X_train, y_train)
test_ds = ToyDataset(X_test, y_test)
print(len(train_ds))

5


In [18]:
from torch.utils.data import DataLoader
torch.manual_seed(123)
train_loader = DataLoader(
    dataset=train_ds,    
    batch_size=2,
    shuffle=True,         
    num_workers=0    
)
test_loader = DataLoader(
    dataset=test_ds,
    batch_size=2,
    shuffle=False,    
    num_workers=0
)
for idx, (x, y) in enumerate(train_loader):
    print(f"Batch {idx+1}:", x, y)
train_loader = DataLoader(
    dataset=train_ds,
    batch_size=2,
    shuffle=True,
    num_workers=0,
    drop_last=True
)
for idx, (x, y) in enumerate(train_loader):
    print(f"Batch {idx+1}:", x, y)

Batch 1: tensor([[ 2.3000, -1.1000],
        [-0.9000,  2.9000]]) tensor([1, 0])
Batch 2: tensor([[-1.2000,  3.1000],
        [-0.5000,  2.6000]]) tensor([0, 0])
Batch 3: tensor([[ 2.7000, -1.5000]]) tensor([1])
Batch 1: tensor([[-1.2000,  3.1000],
        [-0.5000,  2.6000]]) tensor([0, 0])
Batch 2: tensor([[ 2.3000, -1.1000],
        [-0.9000,  2.9000]]) tensor([1, 0])


In [19]:
torch.manual_seed(123)
model = NeuralNetwork(num_inputs=2, num_outputs=2)   
optimizer = torch.optim.SGD(
    model.parameters(), lr=0.5
)           
num_epochs = 3
for epoch in range(num_epochs): 
    model.train()
    for batch_idx, (features, labels) in enumerate(train_loader):
        logits = model(features)
        loss = F.cross_entropy(logits, labels)
        optimizer.zero_grad()           
        loss.backward()         
        optimizer.step()       
        print(f"Epoch: {epoch+1:03d}/{num_epochs:03d}"
            f" | Batch {batch_idx:03d}/{len(train_loader):03d}"
            f" | Train Loss: {loss:.2f}")
    model.eval()
print(sum(p.numel() for p in model.parameters() if p.requires_grad))

Epoch: 001/003 | Batch 000/002 | Train Loss: 0.75
Epoch: 001/003 | Batch 001/002 | Train Loss: 0.65
Epoch: 002/003 | Batch 000/002 | Train Loss: 0.44
Epoch: 002/003 | Batch 001/002 | Train Loss: 0.13
Epoch: 003/003 | Batch 000/002 | Train Loss: 0.03
Epoch: 003/003 | Batch 001/002 | Train Loss: 0.00
752


In [20]:
model.eval()
with torch.no_grad():
    outputs = model(X_train)
print(outputs)

tensor([[ 2.8569, -4.1618],
        [ 2.5382, -3.7548],
        [ 2.0944, -3.1820],
        [-1.4814,  1.4816],
        [-1.7176,  1.7342]])


In [21]:
torch.set_printoptions(sci_mode=False)
probas = torch.softmax(outputs, dim=1)
print(probas)
predictions = torch.argmax(probas, dim=1)
print(predictions)
predictions = torch.argmax(outputs, dim=1)
print(predictions)

tensor([[    0.9991,     0.0009],
        [    0.9982,     0.0018],
        [    0.9949,     0.0051],
        [    0.0491,     0.9509],
        [    0.0307,     0.9693]])
tensor([0, 0, 0, 1, 1])
tensor([0, 0, 0, 1, 1])


In [22]:
predictions == y_train
torch.sum(predictions == y_train)

tensor(5)

In [23]:
def compute_accuracy(model, dataloader):
    model = model.eval()
    correct = 0.0
    total_examples = 0
    for idx, (features, labels) in enumerate(dataloader):
        with torch.no_grad():
            logits = model(features)
        predictions = torch.argmax(logits, dim=1)
        compare = labels == predictions      
        correct += torch.sum(compare)     
        total_examples += len(compare)
    return (correct / total_examples).item()
print(compute_accuracy(model, train_loader))

1.0


In [24]:
torch.save(model.state_dict(), "model.pth")
model = NeuralNetwork(2, 2) 
model.load_state_dict(torch.load("model.pth"))

/tmp/ipykernel_2014/1903278102.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("model.pth"))


<All keys matched successfully>

In [25]:
print(torch.cuda.is_available())
tensor_1 = torch.tensor([1., 2., 3.])
tensor_2 = torch.tensor([4., 5., 6.])
print(tensor_1 + tensor_2)
tensor_1 = tensor_1.to("cuda")
tensor_2 = tensor_2.to("cuda")
print(tensor_1 + tensor_2)

True
tensor([5., 7., 9.])
tensor([5., 7., 9.], device='cuda:0')


In [28]:
# Training loop on GPU
torch.manual_seed(123)
model = NeuralNetwork(num_inputs=2, num_outputs=2)
device = torch.device("cuda")     
model = model.to(device)         
optimizer = torch.optim.SGD(model.parameters(), lr=0.5)
num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    for batch_idx, (features, labels) in enumerate(train_loader):
        features, labels = features.to(device), labels.to(device)  
        logits = model(features)
        loss = F.cross_entropy(logits, labels) # Loss function
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print(f"Epoch: {epoch+1:03d}/{num_epochs:03d}"
            f" | Batch {batch_idx:03d}/{len(train_loader):03d}"
            f" | Train/Val Loss: {loss:.2f}")
    model.eval()

Epoch: 001/003 | Batch 000/002 | Train/Val Loss: 0.75
Epoch: 001/003 | Batch 001/002 | Train/Val Loss: 0.65
Epoch: 002/003 | Batch 000/002 | Train/Val Loss: 0.44
Epoch: 002/003 | Batch 001/002 | Train/Val Loss: 0.13
Epoch: 003/003 | Batch 000/002 | Train/Val Loss: 0.03
Epoch: 003/003 | Batch 001/002 | Train/Val Loss: 0.00
